In [1]:
import xarray as xr
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore") #ignora runtime warning por causa de nan na hora da correlacao

from skill import recorte
from skill import clima
from skill import config

# Configurando diretorios

In [2]:
dir_dados = config.dir_dados
dir_shp = config.dir_shp

# Abrindo arquivos

In [3]:
gpcc = xr.open_dataset(f'{dir_dados}/precip.comb.v2020to2019-v2020monitorafter.total.nc')
br_madeira = gpd.read_file(f'{dir_shp}/brasil_madeira.shp')

clima_cam = xr.open_dataset(f'{dir_dados}/cam3_clima.nc')   
clima_cam['chuva'] = (clima_cam.PRECC + clima_cam.PRECSH + clima_cam.PRECL)*3600*24*30*1000 #cria chuva total com as variaveis do dataset

## Abrindo arquivo de previsão p/ utilizar lat e lon p/ interpolacao da climatologia e do observado

A climatologia tem mesma resolução que as previsões do CAM. Entretanto, os pontos de grade não são os mesmos. Isso acaba causando problemas na hora de calcular a anomalia, já que a indexação é feita pelo conjunto lat lon.

In [4]:
aux = xr.open_dataset(f'{dir_dados}/cam3_init1.nc')

lat = aux.lat.values
lon = aux.lon.values

In [5]:
clima_cam = recorte.regridder(clima_cam, lat, lon)
clima_cam_br = recorte.recortar(clima_cam, br_madeira)

# Trabalhando com observado

In [6]:
clima_obs = clima.climatologia(gpcc, '1979-01-01', '1999-12-01')

## Interpola para a grade do cam

In [7]:
clima_obs_interp = recorte.regridder(clima_obs, lat, lon)
clima_obs_br = recorte.recortar(clima_obs_interp, br_madeira)

# Ajuste de Escala das climatologias

In [8]:
month_coef = clima_obs_br.precip / clima_cam_br.chuva.groupby('time.month').mean('time')

# Fazendo ponderação

In [10]:
dir_dados.joinpath('cam3_ajuste').mkdir(exist_ok=True)
dir_dados.joinpath('cam3_ajuste', 'com_skill').mkdir(exist_ok=True)
dir_dados.joinpath('cam3_ajuste', 'sem_skill').mkdir(exist_ok=True)

## Utilizando ponderação pelo skill

In [11]:
for i in range(1,13):

    cam = xr.open_dataset(f'{dir_dados}/cam3_init{i}.nc')
    skill = xr.open_dataset(f'{dir_dados}/cam3-anom_corr-init{i}.nc').__xarray_dataarray_variable__
    skill = skill.where(skill>0, 0)

    cam['chuva'] = (cam.PRECC + cam.PRECSH + cam.PRECL)*3600*24*30*1000
    cam_br = recorte.recortar(cam.chuva, br_madeira)

    cam_br = cam_br.assign_coords(month=cam_br.time.dt.month)

    prec_ajuste = cam_br * month_coef
    prec_final = (prec_ajuste * skill) + ((1-skill) * clima_obs_br.precip)

    prec_final.to_netcdf(dir_dados.joinpath('cam3_ajuste', 'com_skill', f'cam3_ajuste_{i}.nc'))

In [12]:
for i in range(1,13):
    
    cam = xr.open_dataset(f'{dir_dados}/cam3_init{i}.nc')

    cam['chuva'] = (cam.PRECC + cam.PRECSH + cam.PRECL)*3600*24*30*1000
    cam_br = recorte.recortar(cam.chuva, br_madeira)

    cam_br = cam_br.assign_coords(month=cam_br.time.dt.month)

    prec_ajuste = cam_br * month_coef

    prec_ajuste.to_netcdf(dir_dados.joinpath('cam3_ajuste', 'sem_skill', f'cam3_ajuste_{i}.nc'))